In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [6]:
# Grouping all the unique characters
vocab = sorted(set(text))
print('Number of unique characters is: {}'.format(len(vocab)))

Number of unique characters is: 65


In [7]:
# Creating a mapping table from unique characters to index
char2idx = {ch:i for i, ch in enumerate(vocab)}

# Creating a mapping from index to unique characters
idx2char = np.array(vocab)

# Representing each character in text as its integer representation
text_as_int = np.array([char2idx[ch] for ch in text])

print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [8]:
print(repr(text[:13]), ' ------ is mapped to ------>>', text_as_int[:13])

'First Citizen'  ------ is mapped to ------>> [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
# The batch method lets us easily convert these individual characters to sequences of the desired size.
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

print(sequences)

i=0
for item in sequences.take(5):
    print('Sequence ', i, ': ', len(repr(''.join(idx2char[item.numpy()]))), ' : ', repr(''.join(idx2char[item.numpy()])))
    i+=1
    

<BatchDataset shapes: (101,), types: tf.int32>
Sequence  0 :  110  :  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
Sequence  1 :  109  :  'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
Sequence  2 :  109  :  "now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
Sequence  3 :  107  :  "ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
Sequence  4 :  109  :  'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
# Now For each sequence,
# duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

def split_input_sequence(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_sequence)

In [12]:
# Print the first example input and target values:

for input_seq, target_seq in dataset.take(1):
    print(repr(''.join(idx2char[input_seq.numpy()])))
    print(repr(''.join(idx2char[target_seq.numpy()])))

# Each entry in dataset is like: (input_Seq, target_Seq)

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Each index of these vectors is processed as a one time step. For the input at time step 0, the model receives the index for "F" and tries to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

# Creating Training Batches

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000
print(dataset)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


# Building the model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

1. tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;
2. tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use an LSTM layer here.)
3. tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [19]:
#lossfunction

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1745534


In [20]:
# compile
model.compile(optimizer='adam', loss=loss)

In [21]:
# Configure checkpoints
# It is an approach where a snapshot of the state of the system is taken in case of system failure. 
# If there is a problem, not all is lost. 
# The checkpoint may be used directly, or used as the starting point for a new run, picking up where it left off.

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [22]:
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 811s 5s/step - loss: 2.6636
Epoch 2/10
172/172 [==============================] - 818s 5s/step - loss: 1.9626
Epoch 3/10
172/172 [==============================] - 822s 5s/step - loss: 1.6953
Epoch 4/10
172/172 [==============================] - 823s 5s/step - loss: 1.5455
Epoch 5/10
172/172 [==============================] - 821s 5s/step - loss: 1.4564
Epoch 6/10
172/172 [==============================] - 829s 5s/step - loss: 1.3970
Epoch 7/10
172/172 [==============================] - 869s 5s/step - loss: 1.3500
Epoch 8/10
172/172 [==============================] - 818s 5s/step - loss: 1.3129
Epoch 9/10
172/172 [==============================] - 854s 5s/step - loss: 1.2783
Epoch 10/10
172/172 [==============================] - 826s 5s/step - loss: 1.2464


# Text Generation

In [43]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


The prediction loop
The following code block generates the text:

1. Begin by choosing a start string, initializing the RNN state and setting the number of characters to generate.

2. Get the prediction distribution of the next character using the start string and the RNN state.

3. Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

4. The RNN state returned by the model is fed back into the model so that it now has more context, instead of only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

In [46]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000
    
    # Converting the intial string into a seq of indices- vectorizing
    input_seq = [char2idx[ch] for ch in start_string]
    # print(input_seq)
    input_seq = tf.expand_dims(input_seq, 0)
    # print(input_seq)

    
    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0
    
    # Here batch size == 1
    model.reset_states()
    
    for i in range(num_generate):
        predictions = model(input_seq)
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_seq = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])
        
    return (start_string + ''.join(text_generated))



In [53]:
print(generate_text(model, start_string=u"CEASER: "))

CEASER: I is the king weeps;
The bride and tear of soath, my lord, belike he?
Our princely kear than stily,
Though all the truth, my pain we'll be thus:
This is his lord, thou wert, theadle with the buttergar,
Tender all roht with heaven and betwitchorate.
Thou hast a fearful meane I of reigns:
Good tentue saw their power?

CORIOLANUS:
I pity my son-gently, thou thens
Come to the world of the burnt-maide the
Butiest had the princess, and be nectless men's,
I kill'd with man 'Gfands.
And I was well discolding in this villany.
Ishiles I will throw from jest, and about me the
e was so deliveridy, I kill thee.

KING RICHARD II:
So one that? upon me, Mortinine,
Look in the king, by wedit of my easting, and
Even so my hand.
Which the are you would my great eyesom of me.
The beirg he nought copixation;
As blained trouble us
Hourly husband's change my guilty passage.

ESCALUS:
Joor heart, hastest die for this sadd touches,
How sworn some home as she be new made themjock,
Or tell thee burnable 